# Introduction


In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import power_grid_model as pgm
from generate_fictional_dataset import generate_fictional_grid, generate_time_series
import time
from pathlib import Path
import tempfile

output_dir = Path(tempfile.gettempdir()) / "pandapower_time_series"
output_dir.mkdir(exist_ok=True)


In [2]:
# summary
summary_df = pd.DataFrame(
    np.zeros(shape=(8, 3), dtype=np.float64), 
    columns=['PGM Linear', 'PGM NR', 'PandaPower NR'],
    index=[
        'Symmetric calculation with solver initialization',
        'Symmetric calculation without solver initialization',
        'Asymmetric calculation with solver initialization',
        'Asymmetric calculation without solver initialization',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

In [3]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = 100
n_node = n_node_per_feeder * n_feeder + 1
n_line = n_node_per_feeder * n_feeder
n_load = n_node_per_feeder * n_feeder

# Pre-cache Library


In [4]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=3,
    n_feeder=2,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp.runpp(fictional_dataset['pp_net'], algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
pgm_model = pgm.PowerGridModel(fictional_dataset['pgm_dataset'])
pgm_result = pgm_model.calculate_power_flow()

# Single Calculation

In [5]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']


## Symmetric

In [6]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM Linear'] = end - start

In [7]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM NR'] = end - start


In [8]:
# first calculation with solver initialization
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PandaPower NR'] = end - start

In [9]:
np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()

1.6255885526561542e-12

In [10]:
np.abs(pp_net.res_line['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

5.341560527227784e-13

## Asymmetric

In [11]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM Linear'] = end - start

In [12]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM NR'] = end - start

In [13]:
# first calculation with solver initialization
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PandaPower NR'] = end - start


In [14]:
np.abs(pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].to_numpy() - pgm_result['node']['u_pu']).max()

5.797067270663092e-10

In [15]:
np.abs(pp_net.res_line_3ph['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

8.19211526392749e-09

# Time Series Calculation

In [16]:
time_series_dataset = generate_time_series(
    fictional_dataset,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max
)

pgm_update_dataset = time_series_dataset['pgm_update_dataset']
pp_dataset = time_series_dataset['pp_dataset']
time_steps = np.arange(n_step)

for x, y in zip(['p', 'q'], ['mw', 'mvar']):
    for p in ['a', 'b', 'c']:
        name = f'{x}_{p}_{y}'
        _ = pp.control.ConstControl(
            pp_net, element='asymmetric_load', 
            element_index=pp_net.asymmetric_load.index, variable=name, 
            data_source=pp_dataset[name], profile_name=pp_net.asymmetric_load.index)


## Symmetric

In [17]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM Linear'] = end - start

In [18]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM NR'] = end - start

In [19]:
_ = pp.timeseries.OutputWriter(
    pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',',
    log_variables=[
        ('res_bus', 'vm_pu'),
        ('res_line', 'loading_percent'),
    ]
)

start = time.time()
pp.timeseries.run_timeseries(
    pp_net, run=pp.runpp, time_steps=time_steps,
    calculate_voltage_angles=True, distributed_slack=True
)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PandaPower NR'] = end - start

Progress: |██████████████████████████████████████████████████| 100.0% Complete



In [20]:
pp_u_pu = pd.read_csv(output_dir / 'res_bus' / 'vm_pu.csv', index_col=0).to_numpy()
np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()

1.7075230118734908e-12

In [21]:
pp_loading = pd.read_csv(output_dir / 'res_line' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
np.abs(pp_loading - pgm_result['line']['loading']).max()


7.880363028789361e-13

## Asymmetric

In [22]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM Linear'] = end - start

In [23]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM NR'] = end - start

In [24]:
del pp_net.output_writer
ow = pp.timeseries.OutputWriter(
    pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',')
ow.log_variable('res_bus_3ph', 'vm_a_pu', index=pp_net.bus.index)   
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=pp_net.bus.index)  
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=pp_net.bus.index)  
ow.log_variable('res_line_3ph', 'loading_percent', index=pp_net.line.index)  


# run
start = time.time()
pp.timeseries.run_timeseries(
    pp_net, run=pp.runpp_3ph, time_steps=time_steps,
    calculate_voltage_angles=True, distributed_slack=True
)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PandaPower NR'] = end - start

Progress: |██████████████████████████████████████████████████| 100.0% Complete



In [25]:
pp_u_pu = []
for p in ['a', 'b', 'c']:
    pp_u_pu.append(pd.read_csv(output_dir / 'res_bus_3ph' / f'vm_{p}_pu.csv', index_col=0).to_numpy())
pp_u_pu = np.stack(pp_u_pu, axis=-1)
np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()

4.927085406336573e-10

In [26]:
pp_loading = pd.read_csv(output_dir / 'res_line_3ph' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
np.abs(pp_loading - pgm_result['line']['loading']).max()

2.435985968496368e-09

# Summary of Performace

In [27]:
summary_df

,PGM Linear,PGM NR,PandaPower NR
Symmetric calculation with solver initialization,0.002048,0.004722,0.024580
Symmetric calculation without solver initialization,0.001559,0.004258,0.016900
Asymmetric calculation with solver initialization,0.003144,0.012902,0.190064
Asymmetric calculation without solver initialization,0.003568,0.009747,0.200694
Time series symmetric calculation,0.099407,0.430629,2.800712
Time series asymmetric calculation,0.252091,1.050756,35.543765
N-1 symmetric calculation,0.000000,0.000000,0.000000
N-1 asymmetric calculation,0.000000,0.000000,0.000000


In [28]:
pp_net

This pandapower network includes the following parameter tables:
   - bus (1001 elements)
   - asymmetric_load (1000 elements)
   - ext_grid (1 element)
   - line (1000 elements)
   - controller (6 elements)
   - output_writer (1 element)
 and the following results tables:
   - res_bus (1001 elements)
   - res_line (1000 elements)
   - res_ext_grid (1 element)
   - res_asymmetric_load (1000 elements)
   - res_bus_3ph (1001 elements)
   - res_line_3ph (1000 elements)
   - res_ext_grid_3ph (1 element)
   - res_asymmetric_load_3ph (1000 elements)